In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC

target = 'moved_after_2019'
idx = 'user_id'
root = '/Users/senaemirler/garanti-bbva-data-camp/'


In [2]:
df_train = pd.read_csv(os.path.join(root, 'train_users.csv'))
df_test  = pd.read_csv(os.path.join(root, 'test_users.csv'))
df_subm  = pd.read_csv(os.path.join(root, 'submission.csv'))
df_lang  = pd.read_csv(os.path.join(root, 'languages.csv'))
df_edu   = pd.read_csv(os.path.join(root, 'education.csv'))
df_skill = pd.read_csv(os.path.join(root, 'skills.csv'))
df_exp   = pd.read_csv(os.path.join(root, 'work_experiences.csv'))

In [3]:
num_missing = df_lang['language'].isna().sum()
print(f"'proficiency' sütununda {num_missing} boş değer var.")


'proficiency' sütununda 0 boş değer var.


In [4]:
df_edu.fillna({'fields_of_study':'other'}, inplace=True)

In [5]:
df_edu['fields_of_study'] = df_edu['fields_of_study'].apply(lambda x: x.lower() if type(x) == str else x)


In [ ]:
counts = df_edu['fields_of_study'].value_counts()
df_edu['fields_of_study'] = df_edu['fields_of_study'].apply(lambda x: 'other' if counts[x] == 1 else x)


In [ ]:
# 'fields_of_study' sütunundaki değerlerin sayısını sayma
counts = df_edu['fields_of_study'].value_counts()

# 20'den az tekrarlanan değerleri seçme
rare_values = counts[counts < 25].index.tolist()

# 'other' olarak değiştirme
df_edu.loc[df_edu['fields_of_study'].isin(rare_values), 'fields_of_study'] = 'other'

In [ ]:
import re

df_edu['fields_of_study'] = df_edu['fields_of_study'].apply(lambda x: 'other' if re.search('numerical|öğrenci|hazırlık|öğrenci|i̇ngilizce hazırlık|öğrenci|eşit ağırlık|sayisal|mf|sayısal|mba|bachelor|phd|master|lise|high school', str(x).lower()) else x)

In [8]:
unique_values_location = df_skill['skill'].unique()

location_list = []
for value in unique_values_location:
    if 'endüstri' in value.lower():
        location = value.strip().title()
        location_list.append(location)

print(location_list)


['Endüstriyel Otomasyon', 'Endüstriyel Otomasyon', 'Endüstriyel Kontrol', 'Endüstriyel Tasarım', 'Endüstri Mühendisliği', 'Endüstriyel Elektronik', 'Tekstil Endüstrisi', 'Endüstri 4.0', 'Yiyecek Endüstrisi', 'Endüstriyel Malzeme', 'Endüstriyel Malzeme', 'Endüstri 4.0', 'Yarı İletken Endüstrisi', 'Endüstriyel Hijyen', 'Endüstri', 'Endüstriyel Sektör', 'Petrol Ve Doğal Gaz Endüstrisi', 'Endüstriyel Otomasyon', 'Atıksu Arıtma Tesis Sorumlusu  (Kentsel-Endüstriyel) (A Sınıfı)', 'Endüstriyel Robot Proğramcılığı', 'Endüstri 4', 'Endüstriyel Ve Büyük Tüketimli Alanlarda Doğalgaz Uygulamaları', 'Endüstriyel Süreç Dijitalleştirme', 'Endüstriyel Network']


In [ ]:
fields_of_study_conv = {
    "(?i).*(otomotiv).*": "otomotiv sektörü",
    "(?i).*(informatik|siber|bilgi teknolojisi|game|bilgi teknolojileri|information security|i̇nternet|android|cyber|internet|yazılım|bilişim|ctis|bi̇li̇şi̇m|informatics|game design|data|computational|yazilim|software|information systems|bilgisayar|computer|sofware|veri|web|bilişim teknolojileri|yönetim bilişim|bilişim sistemleri|grafik tasarım|oyun|information technology|information technologies|management information|yönetim bilgi sistemleri).*": "yazılım sektörü",
    "(?i).*(elektrik|electric|elektronik|electrical|electronic|elektrotech|elektric|mekatronik|mechatronic).*": "elektrik sektörü",
    "(?i).*(telekom|telecom).*": "telekom sektörü",
    "(?i).*(sinyal|signal).*": "sinyal işleme",
    "(?i).*(mechanical|mekanik|control|kontrol ve otomasyon|malzeme).*": "mekanik sektörü",
    "(?i).*(civil|industrial engineer|endüstri mühe).*": "endüstri mühendisliği",
    "(?i).*(mis|yönetim ve|işletme|i̇şletme|business|isletme|political|bilgi yönetimi|technology management|engineering management|kamu yönetimi|mühendislik yönetimi|management|kalite yönetimi|yönetim).*": "işletme ve yönetim",
    "(?i).*(economic|economy|ekonomi|maliye|iktisat|i̇ktisat|economet|finance|finans|ekonometri).*": "ekonomi ve finans ",
    "(?i).*(math|matematik|i̇statistik|istatistik).*": "matematik",
    "(?i).*(makine|makina|machine).*": "makine mühendisliği",
    "(?i).*(chemical|kimya|chemistry).*": "kimya sektörü",
    "(?i).*(i̇nşaat|inşaat|construction|constuc|i̇nşaat|mimar|architect).*": "inşaat sektörü",
    "(?i).*(biyomedikal|doctor|pharmacy|eczacılık|doktor|psychology|tıp|medicine|sağlık|health|veter).*": "sağlık sektörü",
    "(?i).*(sanat|art|graphic design).*": "sanat",
    "(?i).*(education|teacher|öğrenim|öğretim|eğitim|history|philosophy|tarih|felsefe|geography).*": "eğitim öğretim",
    "(?i).*(turk|türk|turkish|türkçe|edebiyat|dilbilim).*": "türkçe",
    "(?i).*(advert|reklam|communication design and management|reklam).*": "reklam",
    "(?i).*(yabancı dil|english|ingilizce|ingiliz|ing|eng|mütercim|foreign languages|i̇ngilizce).*": "ingilizce",
    "(?i).*(physics|fizik|fen|astronomy|biyoloji|biology|biyoteknoloji|physical).*": "fen bilimleri",
    "(?i).*(law|hukuk|adalet|justice).*": "hukuk",
    "(?i).*(trade|pazarlama|ticaret|ticaret ).*": "ticaret",
    
}

df_edu.loc[:, 'fields_of_study'] = df_edu.loc[:, 'fields_of_study'].replace(fields_of_study_conv, regex=True)


In [9]:
# get value counts of 'location' column and sort by count
value_counts = df_lang['language'].value_counts().sort_values(ascending=False)
num_unique = df_lang['language'].nunique()

# print the number of unique values
print(f"There are {num_unique} unique values in the 'fields_of_study' column.")

unique_values_degree = df_lang['language'].unique()

# iterate through the value counts series and print the value and its count
for value, count in value_counts.items():
    print(f"{value}: {count}")


There are 513 unique values in the 'fields_of_study' column.
İngilizce: 20702
English: 14863
Türkçe: 14007
Turkish: 8261
Almanca: 3732
German: 3098
French: 934
Spanish: 840
İspanyolca: 837
Fransızca: 757
ingilizce: 584
Rusça: 537
Russian: 520
Arabic: 519
Arapça: 385
Italian: 351
İtalyanca: 333
İngilizce, Orta (1100-1500): 330
Japanese: 276
Deutsch: 203
Japonca: 161
Azerbaijani: 152
İngilizce : 152
İngilizce, Eski (yaklaşık 450-1100): 126
Türkçe, Osmanlıca (1500-1928): 122
Chinese: 94
Persian: 88
Korece: 77
Kurdish: 70
Korean: 70
Dutch: 68
Çince: 66
Ingilizce: 65
Bulgarian: 65
Lehçe: 64
Azerice: 64
Urdu: 59
Greek: 55
Bulgarca: 52
Kürtçe: 50
Polish: 43
Swedish: 41
Portuguese: 39
Latince: 38
Turkish Sign Language: 38
Albanian: 37
Español: 36
Armenian: 35
english: 35
English : 34
Turkmen: 33
Turkish, Ottoman (1500-1928): 32
Almanca, Orta Yüksek (yaklaşık 1050-1500): 32
Français: 32
Germany: 32
İNGİLİZCE: 30
Portekizce: 30
Kazakh: 29
Farsça: 28
Sırpça: 27
Boşnakça: 26
Yunanca: 23
Sign Langu

In [ ]:
df_edu.loc[df_edu['school_name'].str.contains("kocaeli|gebze", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "kocaeli üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("fırat|firat", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "fırat üniversitesi"
df_edu.loc[df_edu['school_name'].str.contains("i̇stanbul technical|i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu|istanbul technical university", na=False) & ~df_edu['school_name'].str.contains("other_string"), 'school_name'] =  "istanbul teknik üniversitesi"



In [ ]:
school_name_conv = {
    
    "(?i).*(hacettepe).*": "hacettepe üniversitesi",
    "(?i).*(i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu|i̇stanbul technical|i̇stanbul teknik|i̇stanbul tecnic|itü|ıtu|ıtü|itu|istanbul technical university).*": "itü",
    "(?i).*(yıldız teknik|ytü|ytu|yildiz|yıldız).*": "ytü",
    "(?i).*(sakarya).*": "sakarya üniversitesi",
    "(?i).*(anadolu university|anadolu üniversitesi).*": "anadolu üniversitesi",
    "(?i).*(marmara).*": "marmara üniversitesi",
    "(?i).*(fırat|firat).*": "celal üniversitesi",    
    "(?i).*(kocaeli|gebze).*": "kocaeli üniversitesi",
    "(?i).*(trakya).*": "trakya üniversitesi",
    "(?i).*(ankara üniversitesi|ankara universitesi|ankara univercity|ankara university).*": "ankara üniversitesi",
    "(?i).*(beykent).*": "beykent üniversitesi",
    "(?i).*(selçuk).*": "selçuk üniversitesi",
    "(?i).*(bahcesehir|bahçeşehir).*": "bahcesehir üniversitesi",
    "(?i).*(eylül|eylul).*": "eylül üniversitesi",
    "(?i).*(karadeniz).*": "karadeniz üniversitesi",
    "(?i).*(osmangazi).*": "osmangazi üniversitesi",
    "(?i).*(gazi|gazı).*": "gazi|gazı üniversitesi",
    "(?i).*(istanbul üniversitesi|istanbul university|i̇stanbul üniversitesi).*": "istanbul üniversitesi",
    "(?i).*(ege).*": "ege üniversitesi",
    "(?i).*(uludağ|uludag).*": "uludağ üniversitesi",
    "(?i).*(celal).*": "celal üniversitesi",
    "(?i).*(atılım|atilim).*": "atılım üniversitesi",
    "(?i).*(galatasaray).*": "galatasaray üniversitesi",
    "(?i).*(başkent|baskent).*": "başkent üniversitesi",
    "(?i).*(eskişehir|eskisehir).*": "eskişehir üniversitesi",
    "(?i).*(atatürk|ataturk|ataturk).*": "atatürk üniversitesi",
    "(?i).*(istanbul bilgi|i̇stanbul bilgi).*": "istanbul bilgi üniversitesi",
    "(?i).*(ahmet yesevi).*": "ahmet yesevi üniversitesi",
    "(?i).*(koç|koc).*": "koç üniversitesi",
    "(?i).*(mugla|muğla).*": "muğla üniversitesi",
    "(?i).*(sabanci|sabancı).*": "sabancı üniversitesi",
    "(?i).*(yeditepe).*": "yeditepe üniversitesi",
    "(?i).*(metu|odtü|middle east|orta doğu| orta dogu).*": "orta doğu teknik üniversitesi",
    "(?i).*(tobb).*": "tobb üniversitesi",
    "(?i).*(çukurova|cukurova).*": "çukurova üniversitesi",
    "(?i).*(erciyes).*": "erciyes üniversitesi",
    "(?i).*(bilkent üniversitesi|bilkent university|bilkent).*": "bilkent üniversitesi",
    "(?i).*(bosphorus|boğaziçi university|boğaziçi üniversitesi|boun|boün|bogaz).*": "boğaziçi üniversitesi",
    "(?i).*(süleyman|suleyman).*": "süleyman üniversitesi", 
    "(?i).*(duzce|düzce|düzce).*": "düzce üniversitesi",
    "(?i).*(pamukkale).*": "pamukkale üniversitesi",
    "(?i).*(çankaya|cankaya).*": "çankaya üniversitesi",
    "(?i).*(maltepe).*": "maltepe üniversitesi",
    "(?i).*(karabuk|karabük).*": "karabük üniversitesi",
    "(?i).*(selcuk|selçuk).*": "pamukkale üniversitesi",
    "(?i).*(okan).*": "okan üniversitesi",
    "(?i).*(özyeğin|ozyegin|özyeğin university).*": "özyeğin üniversitesi",
    "(?i).*(i̇stanbul aydın|istanbul aydın|i̇stanbul aydin).*": "istanbul aydın üniversitesi",
    "(?i).*(zaim).*": "zaim üniversitesi",
    "(?i).*(katip çelebi|katip celebi).*": "katip çelebi üniversitesi",
    "(?i).*(mimar sinan).*": "mimar sinan üniversitesi",
    "(?i).*(yaşar|yasar).*": "yaşar üniversitesi",
    "(?i).*(çanakkale|canakkale|onsekiz mart).*": "çanakkale onsekiz mart üniversitesi",
    "(?i).*(selcuk|selçuk).*": "selçuk üniversitesi",
    "(?i).*(balıkesir|balikesir).*": "balıkesir üniversitesi",
    "(?i).*(i̇zmir ekonomi|izmir ekonomi).*": "izmir ekonomi üniversitesi",
    "(?i).*(haliç|halic).*": "haliç üniversitesi",
    "(?i).*(fatih).*": "fatih üniversitesi",
    "(?i).*(abant).*": "abant üniversitesi",
    "(?i).*(cerrahpaşa).*": "istanbul üniversitesi",
    "(?i).*(doğu akdeniz|dogu akdeniz|eastern mediterranean ).*": "doğu akdeniz üniversitesi",
    "(?i).*(i̇nönü|inönü|ınonu).*": "akdeniz üniversitesi",
    "(?i).*(dumlup).*": "dumlupınar üniversitesi",
    "(?i).*(namık kema|namik kema).*": "namık kema üniversitesi",
    "(?i).*(i̇stanbul gelişim|istanbul gelişim|istanbul gelisim).*": "istanbul gelişim üniversitesi",
    "(?i).*(istanbul kultur|istanbul kültür).*": "istanbul kultur üniversitesi",
    "(?i).*(bilecik).*": "bilecik üniversitesi",
    "(?i).*(üsküdar|uskudar).*": "üsküdar üniversitesi",
    "(?i).*(iskenderun|i̇skenderun).*": "iskenderun üniversitesi",
    "(?i).*(stanford).*": "stanford üniversitesi",
    "(?i).*(karatay).*": "karatay üniversitesi",
    "(?i).*(kadir has).*": "kadir has üniversitesi",
    "(?i).*(sivas).*": "sivas üniversitesi",
    "(?i).*(işık|ışık).*": "ışık üniversitesi",
    "(?i).*(konya technica|konya teknik).*": "konya teknik üniversitesi",
    
}

df_edu.loc[:, 'school_name'] = df_edu.loc[:, 'school_name'].replace(school_name_conv, regex=True)
